# Block Zone Simulator

This notebook / repo will be used to play around with block zone configurations and run simulations to determine the theoretical and practical capacity of an attraction.

In [5]:
# operate in seconds

blocks = {
    'station': {
        'next_block': 'lift 1',
        'seconds_to_reach_block': 8,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': None,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': True,
        'hold_time': 38
    },
    'lift 1': {
        'next_block': 'gravity 1',
        'seconds_to_reach_block': 18,
        'seconds_to_clear_from_held': 8,
        'seconds_to_clear_block_in_motion': 7,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None
    },
    'gravity 1': {
        'next_block': 'lift 2',
        'seconds_to_reach_block': 30,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': 3,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None
    },
    'lift 2': {
        'next_block': 'gravity 2',
        'seconds_to_reach_block': 20,
        'seconds_to_clear_from_held': 8,
        'seconds_to_clear_block_in_motion': 7,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None
    },
    'gravity 2': {
        'next_block': 'final block 1',
        'seconds_to_reach_block': 22,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': 3,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None
    },
    'final block 1': {
        'next_block': 'station',
        'seconds_to_reach_block': 8,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': 3,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None
    }
}

# put trains at station, then lift 1, then lift 2, then final block 1
trains = {
    'train 1': {
        'current_block': 'station',
        'next_block': 'lift 1',
        'seconds_to_reach_block': 0,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': 0,
        'seconds_held_at_current_block': 0,
        'total_seconds_held': 0,
        'mandatory_hold_left': 0,
        'current_status': 'held',  # can be 'held', 'before block', 'after block - from held', 'after block - not held'
        'circuits_completed': 0,
        'lead_train': True
    },
    'train 2': {
        'current_block': 'lift 1',
        'next_block': 'gravity 1',
        'seconds_to_reach_block': 0,
        'seconds_to_clear_from_held': 0,
        'seconds_to_clear_block_in_motion': 0,
        'seconds_held_at_current_block': 0,
        'total_seconds_held': 0,
        'mandatory_hold_left': 0,
        'current_status': 'held',
        'circuits_completed': 0,
        'lead_train': False
    },
    'train 3': {
        'current_block': 'lift 2',
        'next_block': 'gravity 2',
        'seconds_to_reach_block': 0,
        'seconds_to_clear_from_held': 0,
        'seconds_to_clear_block_in_motion': 0,
        'seconds_held_at_current_block': 0,
        'total_seconds_held': 0,
        'mandatory_hold_left': 0,
        'current_status': 'held',
        'circuits_completed': 0,
        'lead_train': False
    },
    'train 4': {
        'current_block': 'final block 1',
        'next_block': 'station',
        'seconds_to_reach_block': 0,
        'seconds_to_clear_from_held': 0,
        'seconds_to_clear_block_in_motion': 0,
        'seconds_held_at_current_block': 0,
        'total_seconds_held': 0,
        'mandatory_hold_left': 0,
        'current_status': 'held',
        'circuits_completed': 0,
        'lead_train': False
    }
}
num_trains = len(trains)
for train in trains:
    blocks[trains[train]['current_block']]['is_occupied'] = True
time = 0
for _ in range(36000):
    trains_advanced = 0
    trains_blocked = 0
    # advance each train if possible
    for train_name in trains:
        curr_block = trains[train_name]['current_block']
        next_block = trains[train_name]['next_block']
        if trains[train_name]['current_status'] == 'held':
            # do a thing because they are held rn
            if trains[train_name]['mandatory_hold_left'] > 0:
                # held and not ready to go
                trains[train_name]['mandatory_hold_left'] -= 1
            else:
                # held and ready to go
                # is the next block ready?
                if blocks[next_block]['is_occupied']:
                    # we cannot go anywhere
                    if not blocks[curr_block]['can_operate_from_stop']:
                        # TODO: signal to all other trains to stop at the next possible block.
                        raise ValueError(f"Train {train_name} halted at block {curr_block}. Ride is now in 101 status.")
                    trains[train_name]['seconds_held_at_current_block'] += 1
                    trains[train_name]['total_seconds_held'] += 1
                    trains_blocked += 1
                else:
                    # we can proceed but from held position
                    # mark next block as occupied, do not mark this one as unoccupied, start countdown for "from held"
                    blocks[next_block]['is_occupied'] = True
                    trains[train_name]['seconds_to_clear_from_held'] = blocks[curr_block]['seconds_to_clear_from_held']
                    trains[train_name]['current_status'] = 'after block - from held'
        # elif/else... means we are in motion.  EITHER: seconds_to_reach_block > 0 (we haven't reached our own block yet), OR
        # seconds_to_clear_from_held > 0 (we were held and were recently released), OR seconds_to_clear_block_in_motion > 0 (we reached
        # our own block but haven't exited it yet).  Only ONE of these should be > 0 at any given time. if all are 0...
        elif trains[train_name]['seconds_to_reach_block'] > 0:
            trains[train_name]['seconds_to_reach_block'] -= 1
        elif trains[train_name]['seconds_to_clear_from_held'] > 0:
            # we were held, decrease this
            trains[train_name]['seconds_to_clear_from_held'] -= 1
        elif trains[train_name]['seconds_to_clear_block_in_motion'] > 0:
            # we are past our own block, never stopped at it either
            trains[train_name]['seconds_to_clear_block_in_motion'] -= 1
        else:
            # 'before block', 'after block - from held', 'after block - not held'
            # we either reached the block OR are ready to exit it
            if trains[train_name]['current_status'] == 'before block':
                # we hadn't reached the block and now we either have to keep moving from motion or stop
                if blocks[next_block]['is_occupied']:
                    # this means another train is there! we cannot proceed.
                    # mark train as held
                    trains[train_name]['current_status'] = 'held'
                    trains_blocked += 1
                    if not blocks[curr_block]['can_operate_from_stop']:
                        # TODO: signal to all other trains to stop at the next possible block.
                        raise ValueError(f"Train {train_name} halted at block {curr_block}. Ride is now in 101 status.")
                else:
                    # we were moving, reached block and are cleared to move forward
                    blocks[next_block]['is_occupied'] = True
                    trains[train_name]['seconds_to_clear_block_in_motion'] = blocks[curr_block]['seconds_to_clear_block_in_motion']
                    trains[train_name]['current_status'] = 'after block - not held'
            # if next block already belongs to us, that means we already reached our block and are proceeding forward
            else:
                # we reached end of block from motion OR stopped, either way... advance to next block
                # we already own the next block, no need to check or alter it
                # release current block
                trains[train_name]['current_block'] = next_block
                trains[train_name]['next_block'] = blocks[next_block]['next_block']
                trains[train_name]['seconds_to_reach_block'] = blocks[next_block]['seconds_to_reach_block']
                trains[train_name]['seconds_held_at_current_block'] = 0
                trains[train_name]['current_status'] = 'before block'
                blocks[curr_block]['is_occupied'] = False
                if next_block == 'station':
                    trains[train_name]['circuits_completed'] += 1
                    #if trains[train_name]['lead_train']:
                    #    print("Lead train completed circuit!")
                trains_advanced += 1
    if trains_blocked == num_trains:
        # we hit gridlock
        raise ValueError(f"Gridlock hit at t={time}!")
    time += 1
                

                

In [143]:
blocks = {
    'station': {
        'next_block': 'lift 1',
        'seconds_to_reach_block': 8,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': None,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': True,
        'hold_time': 38
    },
    'lift 1': {
        'next_block': 'gravity 1',
        'seconds_to_reach_block': 18,
        'seconds_to_clear_from_held': 8,
        'seconds_to_clear_block_in_motion': 7,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None
    },
    'gravity 1': {
        'next_block': 'lift 2',
        'seconds_to_reach_block': 30,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': 3,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None
    },
    'lift 2': {
        'next_block': 'gravity 2',
        'seconds_to_reach_block': 20,
        'seconds_to_clear_from_held': 8,
        'seconds_to_clear_block_in_motion': 7,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None
    },
    'gravity 2': {
        'next_block': 'final block 1',
        'seconds_to_reach_block': 22,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': 3,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None
    },
    'final block 1': {
        'next_block': 'station',
        'seconds_to_reach_block': 8,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': 3,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None
    }
}

class Circuit:
    def __init__(self, blocks, num_trains):
        self.blocks = blocks.copy()
        self.num_complete_blocks = len([b for b in blocks if blocks[b]['can_operate_from_stop']])
        self.num_trains = num_trains
        self.trains = {}
        self.add_trains_to_circuit()
        for train in self.trains:
            self.blocks[self.trains[train]['current_block']]['is_occupied'] = True
        self.time = 0

    def add_trains_to_circuit(self):
        # evenly space out the blocks
        blocks_to_assign = [round(x*self.num_complete_blocks/self.num_trains) for x in range(self.num_trains)]
        valid_blocks = [b for b in self.blocks if blocks[b]['can_operate_from_stop']]
        trains_assigned = 0
        for i in range(self.num_trains):
            train_name = f'train {i}'
            lead_train = (i==0)
            block = valid_blocks[blocks_to_assign[i]]
            train = {
                'current_block': block,
                'next_block': self.blocks[block]['next_block'],
                'seconds_to_reach_block': 0,
                'seconds_to_clear_from_held': 0,
                'seconds_to_clear_block_in_motion': 0,
                'seconds_held_at_current_block': 0,
                'total_seconds_held': 0,
                'mandatory_hold_left': 0,
                'current_status': 'held',
                'circuits_completed': 0,
                'lead_train': lead_train
            }
            self.trains[train_name] = train

    def step(self):
        trains_advanced = 0
        trains_blocked = 0
        # advance each train if possible
        for train_name in self.trains:
            curr_block = self.trains[train_name]['current_block']
            next_block = self.trains[train_name]['next_block']
            if self.trains[train_name]['current_status'] == 'held':
                # do a thing because they are held rn
                if self.trains[train_name]['mandatory_hold_left'] > 0:
                    # held and not ready to go
                    self.trains[train_name]['mandatory_hold_left'] -= 1
                else:
                    # held and ready to go
                    # is the next block ready?
                    if self.blocks[next_block]['is_occupied']:
                        # we cannot go anywhere
                        if not self.blocks[curr_block]['can_operate_from_stop']:
                            # TODO: signal to all other trains to stop at the next possible block.
                            raise ValueError(f"Train {train_name} halted at block {curr_block}. Ride is now in 101 status.")
                        self.trains[train_name]['seconds_held_at_current_block'] += 1
                        self.trains[train_name]['total_seconds_held'] += 1
                        trains_blocked += 1
                    else:
                        # we can proceed but from held position
                        # mark next block as occupied, do not mark this one as unoccupied, start countdown for "from held"
                        self.blocks[next_block]['is_occupied'] = True
                        self.trains[train_name]['seconds_to_clear_from_held'] = self.blocks[curr_block]['seconds_to_clear_from_held']
                        self.trains[train_name]['current_status'] = 'after block - from held'
            # elif/else... means we are in motion.  EITHER: seconds_to_reach_block > 0 (we haven't reached our own block yet), OR
            # seconds_to_clear_from_held > 0 (we were held and were recently released), OR seconds_to_clear_block_in_motion > 0 (we reached
            # our own block but haven't exited it yet).  Only ONE of these should be > 0 at any given time. if all are 0...
            elif self.trains[train_name]['seconds_to_reach_block'] > 0:
                self.trains[train_name]['seconds_to_reach_block'] -= 1
            elif self.trains[train_name]['seconds_to_clear_from_held'] > 0:
                # we were held, decrease this
                self.trains[train_name]['seconds_to_clear_from_held'] -= 1
            elif self.trains[train_name]['seconds_to_clear_block_in_motion'] > 0:
                # we are past our own block, never stopped at it either
                self.trains[train_name]['seconds_to_clear_block_in_motion'] -= 1
            else:
                # 'before block', 'after block - from held', 'after block - not held'
                # we either reached the block OR are ready to exit it
                if self.trains[train_name]['current_status'] == 'before block':
                    # we hadn't reached the block and now we either have to keep moving from motion or stop
                    if self.blocks[curr_block]['mandatory_hold']:
                        # we reached station (or show scene? transfer track? etc... and must pause
                        self.trains[train_name]['current_status'] = 'held'
                        self.trains[train_name]['mandatory_hold_left'] = self.blocks[curr_block]['hold_time']
                    elif self.blocks[next_block]['is_occupied']:
                        # this means another train is there! we cannot proceed.
                        # mark train as held
                        self.trains[train_name]['current_status'] = 'held'
                        trains_blocked += 1
                        if not self.blocks[curr_block]['can_operate_from_stop']:
                            # TODO: signal to all other trains to stop at the next possible block.
                            raise ValueError(f"Train {train_name} halted at block {curr_block}. Ride is now in 101 status.")
                    else:
                        # we were moving, reached block and are cleared to move forward
                        self.blocks[next_block]['is_occupied'] = True
                        self.trains[train_name]['seconds_to_clear_block_in_motion'] = self.blocks[curr_block]['seconds_to_clear_block_in_motion']
                        self.trains[train_name]['current_status'] = 'after block - not held'
                # if next block already belongs to us, that means we already reached our block and are proceeding forward
                else:
                    # we reached end of block from motion OR stopped, either way... advance to next block
                    # we already own the next block, no need to check or alter it
                    # release current block
                    self.trains[train_name]['current_block'] = next_block
                    self.trains[train_name]['next_block'] = self.blocks[next_block]['next_block']
                    self.trains[train_name]['seconds_to_reach_block'] = self.blocks[next_block]['seconds_to_reach_block']
                    self.trains[train_name]['seconds_held_at_current_block'] = 0
                    self.trains[train_name]['current_status'] = 'before block'
                    self.blocks[curr_block]['is_occupied'] = False
                    if next_block == 'station':
                        self.trains[train_name]['circuits_completed'] += 1
                        #if trains[train_name]['lead_train']:
                        #    print("Lead train completed circuit!")
                    trains_advanced += 1
        if trains_blocked == self.num_trains:
            # we hit gridlock
            raise ValueError(f"Gridlock hit at t={time}!")
        self.time += 1

circuit = Circuit(blocks, num_trains=4)

In [144]:
# run the sim
for _ in range(36000):
    circuit.step()

In [145]:
#trains
circuit.trains

{'train 0': {'current_block': 'gravity 2',
  'next_block': 'final block 1',
  'seconds_to_reach_block': 20,
  'seconds_to_clear_from_held': 0,
  'seconds_to_clear_block_in_motion': 0,
  'seconds_held_at_current_block': 0,
  'total_seconds_held': 8155,
  'mandatory_hold_left': 0,
  'current_status': 'before block',
  'circuits_completed': 143,
  'lead_train': True},
 'train 1': {'current_block': 'final block 1',
  'next_block': 'station',
  'seconds_to_reach_block': 0,
  'seconds_to_clear_from_held': 0,
  'seconds_to_clear_block_in_motion': 0,
  'seconds_held_at_current_block': 6,
  'total_seconds_held': 8181,
  'mandatory_hold_left': 0,
  'current_status': 'held',
  'circuits_completed': 143,
  'lead_train': False},
 'train 2': {'current_block': 'station',
  'next_block': 'lift 1',
  'seconds_to_reach_block': 0,
  'seconds_to_clear_from_held': 0,
  'seconds_to_clear_block_in_motion': 0,
  'seconds_held_at_current_block': 0,
  'total_seconds_held': 8181,
  'mandatory_hold_left': 24,
  '

In [146]:
# question - what percent of the sim run time did each train sit idle?
for train_name in circuit.trains:
    print(round(100*circuit.trains[train_name]['total_seconds_held'] / time, 2))
    
# if we set run time to 1 million seconds, they all converge to 17.65% idle.

22.65
22.73
22.73
22.64


In [141]:
blocks

{'station': {'next_block': 'lift 1',
  'seconds_to_reach_block': 8,
  'seconds_to_clear_from_held': 6,
  'seconds_to_clear_block_in_motion': None,
  'is_occupied': True,
  'can_operate_from_stop': True,
  'mandatory_hold': True,
  'hold_time': 38},
 'lift 1': {'next_block': 'gravity 1',
  'seconds_to_reach_block': 18,
  'seconds_to_clear_from_held': 8,
  'seconds_to_clear_block_in_motion': 7,
  'is_occupied': True,
  'can_operate_from_stop': True,
  'mandatory_hold': False,
  'hold_time': None},
 'gravity 1': {'next_block': 'lift 2',
  'seconds_to_reach_block': 30,
  'seconds_to_clear_from_held': 6,
  'seconds_to_clear_block_in_motion': 3,
  'is_occupied': True,
  'can_operate_from_stop': True,
  'mandatory_hold': False,
  'hold_time': None},
 'lift 2': {'next_block': 'gravity 2',
  'seconds_to_reach_block': 20,
  'seconds_to_clear_from_held': 8,
  'seconds_to_clear_block_in_motion': 7,
  'is_occupied': True,
  'can_operate_from_stop': True,
  'mandatory_hold': False,
  'hold_time': No

In [147]:
num_riders_per_train = 24
avg_cycles_completed = sum([circuit.trains[train]['circuits_completed'] for train in circuit.trains]) / len(circuit.trains)
avg_cycles_per_hour = avg_cycles_completed * 3600 / circuit.time
total_cycles_per_hour = avg_cycles_per_hour * len(circuit.trains)
total_hourly_capacity = num_riders_per_train * total_cycles_per_hour
total_hourly_capacity # 463 (1) 928.8 (2) 1394 (3) 1378 (4)

1377.6